# Model Building

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import seaborn as sns

In [2]:
LOGINS_FILENAME = '../data/rba-dataset.csv'
SAMPLED_IPS_FILENAME = './sample-features-ip.csv'

In [5]:
df_logins = dd.read_csv(LOGINS_FILENAME) 
df_ips = pd.read_csv(SAMPLED_IPS_FILENAME)

In [6]:
df_sample = df_logins[df_logins['IP Address'].isin(df_ips['IP Address'])]
df_sample.head() 

,index,Login Timestamp,User ID,Round-Trip Time [ms],IP Address,Country,Region,City,ASN,User Agent String,Browser Name and Version,OS Name and Version,Device Type,Login Successful,Is Attack IP,Is Account Takeover
0,0,2020-02-03 12:43:30.772,-4324475583306591935,NaN,10.0.65.171,NO,-,-,29695,Mozilla/5.0 (iPhone; CPU iPhone OS 13_4 like ...,Firefox 20.0.0.1618,iOS 13.4,mobile,False,False,False
1,1,2020-02-03 12:43:43.549,-4324475583306591935,NaN,194.87.207.6,AU,-,-,60117,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile 46.0.2490,Android 4.1,mobile,False,False,False
3,3,2020-02-03 12:43:56.180,-4324475583306591935,NaN,170.39.78.152,US,-,-,393398,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile WebView 85.0.4183,Android 4.1,mobile,False,False,False
4,4,2020-02-03 12:43:59.396,-4618854071942621186,NaN,10.0.0.47,US,Virginia,Ashburn,398986,Mozilla/5.0 (Linux; U; Android 2.2) Build/NMA...,Chrome Mobile WebView 85.0.4183,Android 2.2,mobile,False,True,False
5,5,2020-02-03 12:44:05.160,-4324475583306591935,NaN,209.236.123.126,US,-,-,393398,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile WebView 85.0.4183,Android 4.1,mobile,False,True,False


In [ ]:
df_sample['login_timestamp_dttm'] = dd.to_datetime(df_sample['Login Timestamp'])
df_sample['hour_of_day'] = df_sample['login_timestamp_dttm'].dt.hour
df_sample['is_norway'] = df_sample['country'] == 'NO'
df_sample = pd.get_dummies(df_sample, columns=['Device Type'])

In [7]:
X = df_sample.iloc[:, 4:-1]
y = df_sample['Is Account Takeover']

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

AttributeError: 'DataFrame' object has no attribute 'take'